<a href="https://colab.research.google.com/github/rubymanderna/ML_ECGR5105/blob/main/Final_project/FNN_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [2]:
from sklearn.metrics import accuracy_score, classification_report

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
data = pd.read_csv("/content/drive/ML_ECGR5105/MyDrive/train_dataframe.csv")
removed_column = data.pop('label')
Y = pd.DataFrame({'age group': removed_column})
X = data.values
X.shape

FileNotFoundError: ignored

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)
X.shape

(17846, 23)

In [ ]:
label_mapping = {label: idx for idx, label in enumerate(np.unique(Y['age group']))}
Y = np.array([label_mapping[label] for label in Y['age group']])
X = torch.tensor(X, dtype=torch.float32)
Y.shape

(17846,)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)







In [ ]:
import torch.nn as nn
# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear( 128, 64)
        self.fc5 = nn.Linear(64,32)
        self.fc6 = nn.Linear(32,output_size)
        self.tanh = nn.Tanh()
        self.soft = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)
    def forward(self, x):
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        #x = self.relu(self.fc3(x))
        #x = self.relu(self.fc4(x))
        x = self.soft(self.fc5(x))
        x = self.fc6(x)
        return x

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def compute_class_weights(y):
    class_weights = compute_class_weight(  class_weight = 'balanced', classes = np.unique(y),y =  y)
    return class_weights

# Calculate class weights based on class distribution
class_weights = compute_class_weights(y_train)

# Convert class weights to a PyTorch tensor
class_weights_tensor =  torch.FloatTensor(class_weights).to(device)

# Use weighted cross-entropy loss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)


In [ ]:
input_size = X_train.shape[1]
hidden_size = 64  # You can adjust this based on your problem
output_size = len(np.unique(y_train))
model = SimpleNN(input_size, hidden_size, output_size).to(device)
X_train_tensor = X_train_tensor.to(device)  # Move training data to GPU
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)  # Move test data to GPU
y_test_tensor = y_test_tensor.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 500
batch_size = 32
for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        labels = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            train_outputs = model(X_train_tensor)
            val_outputs = model(X_test_tensor)
            _, train_predicted = torch.max(train_outputs, 1)
            train_accuracy = accuracy_score(y_train, train_predicted.cpu().numpy())
            _, test_predicted = torch.max(val_outputs, 1)
            test_accuracy = accuracy_score(y_test, test_predicted.cpu().numpy())  # Move back to CPU for metrics
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%, Validation Accuracy: {test_accuracy * 100:.2f}%')
        model.train()

Epoch [100/500], Loss: 1.0960, Training Accuracy: 37.55%, Validation Accuracy: 30.92%
Epoch [200/500], Loss: 0.7497, Training Accuracy: 36.37%, Validation Accuracy: 33.55%
Epoch [300/500], Loss: 1.3203, Training Accuracy: 35.11%, Validation Accuracy: 30.92%
Epoch [400/500], Loss: 0.8785, Training Accuracy: 37.71%, Validation Accuracy: 31.58%
Epoch [500/500], Loss: 0.7839, Training Accuracy: 37.35%, Validation Accuracy: 34.87%


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/test1_extracted_features.csv")
removed_column = data.pop('age')
Y = pd.DataFrame({'age group': removed_column})
X = data.values
sc = StandardScaler()
X = sc.fit_transform(X)
label_mapping = {label: idx for idx, label in enumerate(np.unique(Y['age group']))}
y_test = np.array([label_mapping[label] for label in Y['age group']])
X_test= torch.tensor(X, dtype=torch.float32)
X_test_tensor = torch.FloatTensor(X_test)
X_test_tensor = X_test_tensor.to(device)

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test, predicted.cpu().numpy())  # Move back to CPU for metrics
    print(f'Accuracy on the test set: {accuracy * 100:.2f}%')

# Convert predicted labels back to original age group strings
predicted_labels = [label for label, idx in label_mapping.items() if idx in y_test]
print(predicted.cpu().numpy())
print(classification_report(y_test, predicted.cpu().numpy(), target_names=predicted_labels))

Accuracy on the test set: 25.00%
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7]
              precision    recall  f1-score   support

    eighties       0.00      0.00      0.00         1
     fifties       0.00      0.00      0.00        24
    fourties       0.00      0.00      0.00        22
   seventies       0.00      0.00      0.00         5
     sixties       0.00      0.00      0.00         8
       teens       0.00      0.00      0.00         9
    thirties       0.00      0.00      0.00        45
    twenties       0.25      1.00      0.40        38

    accuracy                           0.25       152
   macro avg       0.03      0.12      0.05       152
weighted avg       0.06      0.25      0.10       152



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
